# Tarea 2

Diego Lozoya Morales | 745345

In [11]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
from IPython.display import display

## Ejercicio 1 

Función VaR y ES

In [12]:
def VaR_CVaR(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]

     weights = []
     for i in range(len(n_stocks)):
          weights.append(valor_posicion.iloc[i]/valor_posicion.sum())
          
     rt_assets = data.pct_change().dropna()

     rt_port = rt_assets.mul(weights).sum(axis=1)

     if long:
          var = np.percentile(rt_port,100-conf)
          cvar = rt_port[rt_port < var].mean()
     else:
          var = np.percentile(rt_port,conf)
          cvar = rt_port[rt_port > var].mean()

     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()

     resultados_df = pd.DataFrame({
        "Métrica": ["VaR", "CVaR"],
        "Porcentual (%)": [np.round(var * 100, 4), np.round(cvar * 100, 4)],
        "Monetario ($)": [np.round(var_m, 2), np.round(cvar_m, 2)]
    })
     
     resultados_df["Porcentual (%)"] = resultados_df["Porcentual (%)"].map("{:,.4f}".format) 
     resultados_df["Monetario ($)"] = resultados_df["Monetario ($)"].map("{:,.2f}".format) 

     return resultados_df

In [13]:
data = yf.download(['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA'], start="2020-01-01", end="2023-01-01")['Close'] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] #Ordenar alfabeticamente

long = True #True o False

NC = 95 #[0, 100]

VaR_CVaR(data, positions, NC, long)

[*********************100%***********************]  5 of 5 completed


,Métrica,Porcentual (%),Monetario ($)
0,VaR,-2.89,"-33,087.27"
1,CVaR,-4.63,"-53,145.10"


## Ejercicio 2

Rebalanceo del portafolio

In [14]:
def Rebalanceo(data, positions, conf, long, target_VaR):

     valor_posicion = positions * data.iloc[-1]

     weights = []
     for i in range(len(positions)):
          weights.append(valor_posicion.iloc[i]/valor_posicion.sum())
          
     rt_assets = data.pct_change().dropna()

     rt_port = rt_assets.mul(weights).sum(axis=1)

     if long:
          var = np.percentile(rt_port,100-conf)
          cvar = abs(rt_port[rt_port < var].mean())
          var = abs(var)
     else:
          var = np.percentile(rt_port,conf)
          cvar = abs(rt_port[rt_port > var].mean())
          var = abs(var)

     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()

     if var_m < target_VaR:
          resultados_df = pd.DataFrame({
             "Métrica": ["VaR", "CVaR"],
             "Porcentual (%)": [np.round(var * 100, 4), np.round(cvar * 100, 4)],
             "Monetario ($)": [np.round(var_m, 2), np.round(cvar_m, 2)]
          })
          
          resultados_df["Porcentual (%)"] = resultados_df["Porcentual (%)"].map("{:,.4f}".format) 
          resultados_df["Monetario ($)"] = resultados_df["Monetario ($)"].map("{:,.2f}".format) 

          return resultados_df
     else:
        target = abs(target_VaR/valor_posicion.sum())

        def objective(weights, rt_assets, conf):
            rt_port_optim = rt_assets.dot(weights)
            var = abs(np.percentile(rt_port_optim, 100 - conf))
            return var

        def constraint(weights, rt_assets, target, conf):
            rt_port_optim = rt_assets.dot(weights)
            var = abs(np.percentile(rt_port_optim, 100 - conf))
            return var - target 

        constraints = (
            {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}, 
            {'type': 'eq', 'fun': lambda weights: constraint(weights, rt_assets, target, conf)} 
        )

        bounds = [(0, 1) for _ in range(len(positions))]

        initial_weights = np.ones(len(positions)) / len(positions)

        result = minimize(objective, initial_weights, args=(rt_assets, conf), bounds=bounds, constraints=constraints)

        if result.success:
            optim_weights = result.x

            w_diff = [a - b for a, b in zip(optim_weights, weights)]
            money_diff = np.array(w_diff) * valor_posicion.sum()

            position_diff = np.floor(money_diff / data.iloc[-1])
            position_diff_df = pd.DataFrame({
                "ticker": data.keys(), 
                "Diferencia porcentual (%)": np.array(w_diff) * 100, 
                "Diferencia en acciones": position_diff.values
            })

            if long:
                position_diff_df['Compra/Venta'] = position_diff_df['Diferencia en acciones'].apply(lambda x: 'Comprar' if x > 0 else ('Vender' if x < 0 else 'NA'))
            else:
                position_diff_df['Compra/Venta'] = position_diff_df['Diferencia en acciones'].apply(lambda x: 'Vender' if x > 0 else ('Comprar' if x < 0 else 'NA'))

            new_positions = [a + b for a, b in zip(position_diff, positions)]
            new_positions_df = pd.DataFrame({
                "ticker": data.keys(), 
                "Ponderaciones (%)": optim_weights * 100, 
                "Cantidad de acciones": new_positions
            })

            print(f"Para rebalancear el portafolio y tener un VaR monetario de ${target_VaR:,.2f} se necesitan los siguientes cambios en el portafolio")
            display(position_diff_df)
            print(f"Finalmente el portafolio quedaría con las siguientes posiciones")
            display(new_positions_df)
        else:
            return "La optimización no fue exitosa"

In [15]:
data = yf.download(['AAPL', 'AMD', 'JPM', 'LMT', 'TSLA'], start="2020-01-01", end="2023-01-01")['Close'] #No puedo descargar el ajustado

positions = [2193, 3221, 1231, 761, 1211] #Ordenar alfabeticamente

long = True #True o False

conf = 95 #[0, 100]

target_VaR = 30000 

Rebalanceo(data, positions, conf, long, target_VaR)

[*********************100%***********************]  5 of 5 completed

Para rebalancear el portafolio y tener un VaR monetario de $30,000.00 se necesitan los siguientes cambios en el portafolio


,ticker,Diferencia porcentual (%),Diferencia en acciones,Compra/Venta
0,AAPL,-2.08,-186.00,Vender
1,AMD,-17.89,"-3,167.00",Vender
2,JPM,11.74,"1,063.00",Comprar
3,LMT,8.33,207.00,Comprar
4,TSLA,-0.11,-11.00,Vender


Finalmente el portafolio quedaría con las siguientes posiciones


,ticker,Ponderaciones (%),Cantidad de acciones
0,AAPL,22.51,"2,007.00"
1,AMD,0.31,54.00
2,JPM,25.32,"2,294.00"
3,LMT,38.96,968.00
4,TSLA,12.90,"1,200.00"


## Ejercicio 3

Investigación conceptual: Rebalanceos

### Inciso a)

- **¿Cuál es el objetivo principal del rebalanceo?**

- **¿Por qué es importante implementarlo de manera sistemática?**


### Inciso b)

- **¿Qué decisiones forman parte de una estrategia de rebalanceo?**

- **¿Cuáles son los métodos principales para realizar rebalanceos de forma sistemática?**


### Inciso c)

- **Además del rendimiento, riesgo y objetivos, ¿qué otro factor puede tener un gran impacto en el desempeño de un portafolio al rebalancearlo?**


### Inciso d)

- **¿Qué recomendaciones propone Vanguard para reducir los costos fiscales en un portafolio?**

- **¿Qué es el tax loss harvesting?**
